## Overview of the Course

Here is a brief overall view of the tutorial:

![image]("REConference/Tutorial/3 Preprocessing - Mitra/course.png")



In [25]:
import pandas as pd
import numpy as np

# for handling text
import nltk
%matplotlib inline
import sklearn
import regex as re
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import time
from termcolor import colored

# for plots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px
%matplotlib inline

# for model creation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score

# we specify for the columns to be shown all together without truncation
pd.set_option('display.max_columns', None)

# Row
pd.set_option('display.max_rows', 10)

# path to the data
ALLData_FILE_PATH = r"D:\Dr. Desh\REConference\Tutorial\3 Preprocessing - Mitra\AllData.csv"

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mitra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mitra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Import the file

In [26]:
all_data = pd.read_csv(ALLData_FILE_PATH)
all_data.head(2)

c:\Users\mitra\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning:

Columns (7,8,9,10,11,13,16,18,19,26,27,28,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,70,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,96,98,99,101,102,103,104,105,106,107,108,109,110,111,112,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,139,140,145,146,147,148,149,150,151,152,153,154,155,156,169) have mixed types.Specify dtype option on import or set low_memory=False.



,Unnamed: 0,alias,assigned_to,assigned_to_detail,blocks,cc,cc_detail,cf_backlog,cf_blocking_20,cf_blocking_fennec,cf_blocking_seamonkey21,cf_blocking_thunderbird31,cf_blocking_webextensions,cf_bmo_component_move,cf_cab_review,cf_colo_site,cf_crash_signature,cf_data_science_status,cf_due_date,cf_firefox_messaging_system,cf_fission_milestone,cf_fx_iteration,cf_fx_points,cf_has_regression_range,cf_has_str,cf_last_resolved,cf_locale,cf_platform_rel,cf_qa_whiteboard,cf_rank,cf_status_conduit_push,cf_status_firefox25,cf_status_firefox26,cf_status_firefox27,cf_status_firefox28,cf_status_firefox29,cf_status_firefox33,cf_status_firefox34,cf_status_firefox40,cf_status_firefox41,cf_status_firefox42,cf_status_firefox43,cf_status_firefox44,cf_status_firefox45,cf_status_firefox46,cf_status_firefox47,cf_status_firefox48,cf_status_firefox49,cf_status_firefox50,cf_status_firefox51,cf_status_firefox52,cf_status_firefox53,cf_status_firefox54,cf_status_firefox55,cf_status_firefox56,cf_status_firefox57,cf_status_firefox58,cf_status_firefox59,cf_status_firefox60,cf_status_firefox61,cf_status_firefox62,cf_status_firefox63,cf_status_firefox64,cf_status_firefox65,cf_status_firefox66,cf_status_firefox67,cf_status_firefox67_0_1,cf_status_firefox68,cf_status_firefox69,cf_status_firefox70,cf_status_firefox_esr52,cf_status_firefox_esr60,cf_status_firefox_esr68,cf_status_fxios_v11_0,cf_status_fxios_v11_1,cf_status_fxios_v12_0,cf_status_fxios_v12_1,cf_status_fxios_v12_2,cf_status_fxios_v13_0,cf_status_fxios_v13_1,cf_status_fxios_v13_2,cf_status_fxios_v13_3,cf_status_fxios_v14_0,cf_status_fxios_v14_1,cf_status_fxios_v14_2,cf_status_fxios_v14_3,cf_status_fxios_v15_0,cf_status_fxios_v15_1,cf_status_fxios_v16_0,cf_status_geckoview62,cf_status_geckoview66,cf_status_seamonkey21,cf_status_seamonkey210,cf_status_seamonkey211,cf_status_seamonkey249,cf_status_seamonkey253,cf_status_seamonkey256,cf_status_seamonkey257esr,cf_status_seamonkey258,cf_status_seamonkey259,cf_status_seamonkey263,cf_status_thunderbird_59,cf_status_thunderbird_60,cf_status_thunderbird_61,cf_status_thunderbird_62,cf_status_thunderbird_63,cf_status_thunderbird_64,cf_status_thunderbird_65,cf_status_thunderbird_66,cf_status_thunderbird_67,cf_status_thunderbird_68,cf_status_thunderbird_69,cf_status_thunderbird_70,cf_status_thunderbird_esr60,cf_status_thunderbird_esr68,cf_tracking_bmo_push,cf_tracking_conduit_push,cf_tracking_e10s,cf_tracking_firefox30,cf_tracking_firefox31,cf_tracking_firefox45,cf_tracking_firefox46,cf_tracking_firefox47,cf_tracking_firefox50,cf_tracking_firefox51,cf_tracking_firefox52,cf_tracking_firefox53,cf_tracking_firefox61,cf_tracking_firefox63,cf_tracking_firefox64,cf_tracking_firefox65,cf_tracking_firefox66,cf_tracking_firefox67,cf_tracking_firefox68,cf_tracking_firefox69,cf_tracking_firefox70,cf_tracking_firefox_esr60,cf_tracking_firefox_esr68,cf_tracking_firefox_relnote,cf_tracking_fxios,cf_tracking_geckoview62,cf_tracking_seamonkey249,cf_tracking_seamonkey253,cf_tracking_seamonkey257esr,cf_tracking_seamonkey263,cf_tracking_thunderbird_59,cf_tracking_thunderbird_60,cf_tracking_thunderbird_61,cf_tracking_thunderbird_62,cf_tracking_thunderbird_63,cf_tracking_thunderbird_64,cf_tracking_thunderbird_65,cf_tracking_thunderbird_66,cf_tracking_thunderbird_67,cf_tracking_thunderbird_68,cf_tracking_thunderbird_69,cf_tracking_thunderbird_70,cf_tracking_thunderbird_esr60,cf_tracking_thunderbird_esr68,cf_user_story,cf_webcompat_priority,classification,comment_count,component,creation_time,creator,creator_detail,depends_on,dupe_of,duplicates,flags,groups,id,is_cc_accessible,is_confirmed,is_creator_accessible,is_open,keywords,last_change_time,mentors,mentors_detail,op_sys,platform,priority,product,qa_contact,qa_contact_detail,regressed_by,regressions,resolution,see_also,severity,status,summary,target_milestone,type,url,version,votes,whiteboard
0,0,NaN,nobody@mozilla.org,"{'email': 'nobody@mozilla.org', 'real_name': '...","10276, 104166","['amanforindia@gmail.com', 'andrewm715+bugzill..."

In [27]:
# remove the redundant column
all_data.drop(columns=['Unnamed: 0'], inplace=True)

# drop duplicate ids
all_data.drop_duplicates(subset='id', inplace=True)

# drop columns that only contain null
all_data.dropna(axis=1, inplace=True, how='all')

### Data Exploration

First, we take a look at the data frame size. Then we realize that we don't need all the 198 columns, so we choose only the ones that can be helpful in understanding the data.

However, it is always necessary to look at the documentation for the data from the source website to understand it better

In [28]:
print(f'There are {len(all_data)} records and {len(all_data.columns)} columns in this dataset.')

There are 4215 records and 198 columns in this dataset.


In [29]:
df = all_data[[
        'classification', 'component', 'platform',  'assigned_to', 
        'summary', 'id', 'status', 'is_confirmed', 'votes',
        'severity', 'keywords', 'priority', 'creator', 'type'
        ]]

In [30]:
print(f'There are {len(df)} records and {len(df.columns)} columns in this dataset.')

There are 4215 records and 14 columns in this dataset.


### Visualize the data to understand it better

In the first part we try to categorize the data into groups and count their frequency. These categories will be placed in a data frame along with the number of times each one was seen.

Here is an overview of what the functions in the next cells will do:


In [31]:
# choose a column
col = 'platform'

# create a new dataset by grouping the categories, then choose id to show the count
categories_count = df.groupby(by=col).count().id.sort_values(ascending=False)

# by resetting the index, we create a df
categories_count_df = categories_count.reset_index()

categories_count_df

,platform,id
0,Unspecified,2727
1,All,1228
2,x86,138
3,x86_64,45
4,ARM64,26
5,Other,18
6,Desktop,14
7,ARM,10
8,PowerPC,8
9,HP,1


In [32]:
categories_count_df.rename(columns={'id': 'count'}, inplace=True)
categories_count_df

,platform,count
0,Unspecified,2727
1,All,1228
2,x86,138
3,x86_64,45
4,ARM64,26
5,Other,18
6,Desktop,14
7,ARM,10
8,PowerPC,8
9,HP,1


In [33]:
def create_grouped_df(df, col):
    """
    creates a dataset for counting categories in one field of a df
    """
    series = df.groupby(by=col).count().id.sort_values(ascending=False).reset_index()
    df_ = pd.DataFrame(series)
    df_.rename(columns={'id': 'count'}, inplace=True)

    return df_

In [34]:
def create_plot(df, col):

    # create the dataset for the visualization
    df_ = create_grouped_df(df, col=col)

    # start plotting
    fig = px.bar(df_, x=col, y='count', text='count',
     labels={'x': col, 'y': 'frequency'}, color='count',
     color_continuous_scale='Agsunset_r')

    fig.update_traces(
        texttemplate='%{text:.2f}', textposition='outside',
        textfont_size=10)

    # Change the bar mode
    fig.update_layout(title_text=f'Field {col} types count')
    fig.show()

Many fields have more than 20 categories which can make visualization and depicting the values challenging. To prevent the plot from getting too messy and illegible, we filter the columns with less than or equal to 20 categories.

In [35]:
# a list of columns we want to visalize
visualization_cols = []

# start filtering
for col in df.columns:
    unique_values = df.loc[:, col].unique()

    if len(unique_values)<20 and len(unique_values)>1:
        print('column: ', colored(f'{col}', 'green'))
        print(unique_values)
        visualization_cols.append(col)

column:  classification
['Components' 'Client Software' 'Server Software' 'Other'
 'Developer Infrastructure']
column:  platform
['All' 'x86' 'PowerPC' 'Unspecified' 'x86_64' 'HP' 'ARM' 'Other' 'Desktop'
 'ARM64']
column:  status
['NEW' 'ASSIGNED' 'REOPENED' 'UNCONFIRMED' 'RESOLVED']
column:  is_confirmed
[ True False]
column:  severity
['normal' 'critical' 'minor' 'blocker' 'major' 'trivial']
column:  priority
['P3' '--' 'P5' 'P2' 'P4' 'P1']
column:  type
['enhancement' 'task']


In [36]:
# loop over the columns and visualize them
for col in visualization_cols:
    create_plot(df, col=col)

### Working with textual data

* Tokenization
* Taking a closer look at the summary data
* Plotting the distribution of words with PlotlyExpress

We'll start by looking at the summary column. As we discussed earlier, first, we'll tokenize the text for a more accurate cleaning, then, based on our task, we choose a couple of cleaning functions one after another.


### Tokenization

In [37]:

def tokenize_text(df, input_col='summary', output_col="summary_tokens_list"):
    """
    takes a dataset and name of a column
    then tokenizes the text in the column of that dataset
    """
    tokenized_series = df.loc[:, input_col].apply(lambda t: word_tokenize(t))
    df.loc[:, output_col] = tokenized_series
    return df
    
df = tokenize_text(df)

c:\Users\mitra\anaconda3\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [38]:
df.head(2)

,classification,component,platform,assigned_to,summary,id,status,is_confirmed,votes,severity,keywords,priority,creator,type,summary_tokens_list
0,Components,Networking: HTTP,All,nobody@mozilla.org,Proxy: map HTTP 500 errors to necko errors (so...,2875,NEW,True,13,normal,[],P3,erik@vanderpoel.org,enhancement,"[Proxy, :, map, HTTP, 500, errors, to, necko, ..."
1,Components,LDAP Integration,All,nobody@mozilla.org,[LDAP] Access to a local LDAP server in Off-Li...,2892,NEW,True,4,normal,['helpwanted'],--,lchiang@formerly-netscape.com.tld,enhancement,"[[, LDAP, ], Access, to, a, local, LDAP, serve..."


Let's see how long each document tokens list is.

In [39]:
def add_token_length_to_df(df, col='summary_tokens_list'):

    length_of_tokens = df.loc[:, col].apply(lambda x: len(x))
    df.loc[:, 'tokens_list_length'] = length_of_tokens
    return df
df = add_token_length_to_df(df)
df.head(2)

c:\Users\mitra\anaconda3\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,classification,component,platform,assigned_to,summary,id,status,is_confirmed,votes,severity,keywords,priority,creator,type,summary_tokens_list,tokens_list_length
0,Components,Networking: HTTP,All,nobody@mozilla.org,Proxy: map HTTP 500 errors to necko errors (so...,2875,NEW,True,13,normal,[],P3,erik@vanderpoel.org,enhancement,"[Proxy, :, map, HTTP, 500, errors, to, necko, ...",19
1,Components,LDAP Integration,All,nobody@mozilla.org,[LDAP] Access to a local LDAP server in Off-Li...,2892,NEW,True,4,normal,['helpwanted'],--,lchiang@formerly-netscape.com.tld,enhancement,"[[, LDAP, ], Access, to, a, local, LDAP, serve...",12


In [40]:
fig = px.histogram(df, x="tokens_list_length", color="type",
                    marginal="box",
                   title='Histogram of summary tokens count',
                   labels={'tokens_list_length':'count of tokens list'}, # can specify one label per df column
                   opacity=0.8,)
fig.show()

The goal of the following cell is to create a dataset of all the words in our dataset and count them as they appear in the records.

In [45]:
def count_words(tokens_arrays):
    """
    gets a dictionary and counts the values
    output: a sorted dict
    note: you can also use a bag of words package to do this
    """
    count_dict = {}
    for array_ in tokens_arrays:
        for word in array_:
            try: count_dict[word] +=1
            except: count_dict[word] = 1

    # sort 
    sorted_count_dict = {k:v for k,v in sorted(count_dict.items(), key=lambda item: item[1], reverse=True)}
    
    return sorted_count_dict 

def get_n_key_and_value(n, dict_):

    """
    get the first - most frequent and important -
    words of dictionary 
    """
    keys = [k for (k, v) in dict_.items()][:n]
    values = [v for (k, v) in dict_.items()][:n]

    return keys, values

def convert_tokens_list_to_freq_df(tokens_arrays, n=-1):
    """
    gets the array of tokenized sentences
    output: a sorted data frame with two cols
    the words and their frequency
    """

    dict_ = count_words(tokens_arrays)
    keys, values = get_n_key_and_value(n, dict_)

    df = pd.DataFrame({'words': keys, 'freq': values})

    return df

    
df_words_freq_task = convert_tokens_list_to_freq_df(df[df.type=='task'].cleaned_word_list, n=-1)
df_words_freq_enh = convert_tokens_list_to_freq_df(df[df.type=='enhancement'].cleaned_word_list, n=-1)


Because they have different number of records, and to have a fair comparison, we use percentage as a way of measurement

In [63]:
def take_percentage(df, col='freq'):

    sum = df.loc[:, 'freq'].sum()
    df.loc[:, 'freq'] = df.loc[:, 'freq'].apply(lambda x: x/sum)
    df.rename(columns={'freq': 'freq_ratio'}, inplace=True)
    return df

df_words_freq_enh = take_percentage(df_words_freq_enh)
df_words_freq_task = take_percentage(df_words_freq_task)

In [64]:
df_words_freq_enh.head()

,words,freq_ratio
0,meta,0.034659
1,implement,0.013933
2,add,0.013686
3,support,0.011619
4,use,0.007582


We merge the datasets for overall analysis and visualization. For this, we rename columns to distinguish each set when merging, but we need to keep the words field the same because we want them to be merged on it.

In [65]:

task_word_freq_df = df_words_freq_enh.add_suffix('_enh')
task_word_freq_df.rename(columns={'words_enh': 'words'}, inplace=True)

enh_word_freq_df = df_words_freq_task.add_suffix('_task')
enh_word_freq_df.rename(columns={'words_task': 'words'}, inplace=True)

merged_df = pd.merge(enh_word_freq_df, task_word_freq_df)

In [66]:
merged_df

,words,freq_ratio_task,freq_ratio_enh
0,meta,0.052440,0.034659
1,history,0.022474,0.000788
2,room,0.020976,0.000098
3,add,0.012414,0.013686
4,remove,0.011558,0.006105
...,...,...,...
1146,embedded,0.000214,0.000246
1147,reset,0.000214,0.000295
1148,proposed,0.000214,0.000049
1149,property,0.000214,0.000738


In [68]:
n = 30
merged_df_sample = merged_df[:n]
fig = go.Figure(data=[
    go.Bar(name='Enhancement', x=merged_df_sample.words, y=merged_df_sample.freq_ratio_enh, text=merged_df_sample.freq_ratio_enh, marker_color='#BA0F30'),
    go.Bar(name='Task', x=merged_df_sample.words, y=merged_df_sample.freq_ratio_task, text=merged_df_sample.freq_ratio_task, marker_color='#98D7C2')
])


fig.update_xaxes(tickangle= -45)  
fig.update_traces(
    texttemplate='%{text:.2f}', textposition='outside',
     textfont_size=8)

# Change the bar mode
fig.update_layout(barmode='group', title_text='Comparing the the most frequent words in enhancement and task types')
fig.show()

### Data Cleaning

These are the functions and the road map we follow for cleaning this dataset:

 1. Lowercase the text
 2. remove the numbers
 3. Remove the stopwords
 4. Remove the punctuation

When we go over all the steps, we append the text to a new list and then attach it to the main dataset.


In [69]:
stop_words = set(stopwords.words('english'))

def remove_stopwords(list_of_words):
    filtered_list = [w for w in list_of_words if not w.lower() in stop_words]
    return filtered_list

def check_punct(list_of_words):
    """
    look at the tokenized text. if the token was matched with ( or was only)
    a puncutation, it is redundant and we remove it.
    """
    filtered_list = []
    for word in list_of_words:
        if re.match("[()!><.,`?':\-\[\]_@]", word):
            pass
        else: filtered_list.append(word)
        
    return filtered_list

def remove_numbers(list_of_words):

    filtered_list = []
    for word in list_of_words:
        if re.findall("[0-9]", word):
            pass
        else: filtered_list.append(word)
        
    return filtered_list


def clean(df, input_col='summary_tokens_list', output_col="cleaned_word_list"):
    
    """
    takes a column of textual data 
    outputs a df with cleaned text attached
    """
    text_series = df.loc[:, input_col]
    cleaned_text_series = []
    
    for text in text_series: 
        # go through every record of this dataset
        # then append the cleaned text to a new list
        text = [t.lower() for t in text]
        cleaned_text = remove_numbers(text)
        cleaned_text = remove_stopwords(cleaned_text)
        cleaned_text = check_punct(cleaned_text)
        cleaned_text_series.append(cleaned_text)
        
    df.loc[:, output_col] = cleaned_text_series
    
    return df

df = clean(df)

c:\Users\mitra\anaconda3\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

c:\Users\mitra\anaconda3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [70]:
df.head(2)

,classification,component,platform,assigned_to,summary,id,status,is_confirmed,votes,severity,keywords,priority,creator,type,summary_tokens_list,tokens_list_length,cleaned_word_list
0,Components,Networking: HTTP,All,nobody@mozilla.org,Proxy: map HTTP 500 errors to necko errors (so...,2875,NEW,True,13,normal,[],P3,erik@vanderpoel.org,enhancement,"[Proxy, :, map, HTTP, 500, errors, to, necko, ...",19,"[proxy, map, http, errors, necko, errors, inte..."
1,Components,LDAP Integration,All,nobody@mozilla.org,[LDAP] Access to a local LDAP server in Off-Li...,2892,NEW,True,4,normal,['helpwanted'],--,lchiang@formerly-netscape.com.tld,enhancement,"[[, LDAP, ], Access, to, a, local, LDAP, serve...",12,"[ldap, access, local, ldap, server, off-line, ..."


### Classification

Step .1 Preparing the input

"Term Frequency — Inverse Document Frequency" is referred to as "TF-IDF". This method first, counts the number of words in a collection of documents. Then, count the number of documents that contain this word. Each word is typically given a score to indicate how important it is to the document and corpus. We frequently use TFIDF vectors to classify, rank, or cluster documents or evaluate words and topics.



* t — term (word)
* d — document (set of words)
* N — count of corpus
* corpus — the total document set

`TF-IDF = Term Frequency (TF) * Inverse Document Frequency (IDF)`

`tf(t,d) = count of t in d / number of words in d`

`df(t) = occurrence of t in N documents`

For instance, a word like "is," which is quite common, may appear several times in a document. So, to prevent words such as stop words have high scores and determinant factors, we perform normalization on the frequency value. We divide the frequency with the total number of words in the document.

For starting, we first, join all the cleaned tokens in each record and feed it as a span of text to TFIDF vectorizer.

In [72]:
# we join the cleaned tokens to feed the tfidf algorithm

def join_tokens(df, tokens_arrays_col):
    return [" ".join(df.loc[i, tokens_arrays_col]) for i in range(len(df))]

cleaned_tokens_series = join_tokens(df, 'cleaned_word_list')
df.loc[:, 'cleaned_text'] = cleaned_tokens_series

c:\Users\mitra\anaconda3\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [73]:
tf_idf = TfidfVectorizer()

#applying tf idf to training data
X = tf_idf.fit_transform(df.loc[:, 'cleaned_text']).toarray()

The above column shows you how we create TFIDF vectors, but to get a better understanding of the scores this algorithm assigns to each word, run the following cell and take a look at the output.

In [78]:
def get_tfidf_words_and_array(text_arrays):

    vectorizer = TfidfVectorizer()
    transformed_data = vectorizer.fit_transform(text_arrays).toarray()
    words = vectorizer.get_feature_names_out()
    
    return transformed_data, words

def create_tfidf_df(text_arrays):
    """
    gets the df, converts it into tfidf arrays and words
    then puts them in a dataset
    """

    transformed_data, words = get_tfidf_words_and_array(text_arrays)

    df = pd.DataFrame(data=transformed_data, columns=words).sum().reset_index()

    col_names = ['words', 'tfidf_score_sum']
    default_col_names = df.columns

    # rename whatever the df cols are called to the col_names
    df.rename(columns={default_col_names[i]:col_names[i] for i in range(len(col_names))}, inplace=True)

    df.sort_values(by='tfidf_score_sum', inplace=True, ascending=False)

    return df

    
tfidf_scores = create_tfidf_df(df.loc[:, 'cleaned_text'])
tfidf_scores

,words,tfidf_score_sum
2772,meta,179.486205
2164,implement,78.972014
54,add,77.073370
4678,support,70.306971
3931,remove,50.829238
...,...,...
2831,mix,0.193280
474,boss,0.193280
2962,mysessions,0.193280
535,buddy,0.193280


Step 2. Creating a model that uses the vectors to classify text

In general, Scikit-learn makes machine learning model training simple and generally follows the same pattern. Here we'll use Naive Bayes to classify our vectors. These are the step we'll have to follow:


1. initialize an instance of the model
2. fit the model to the train data
3. predict the test data 
4. compute scores for both train and test sets.


To put it very briefly, it is worth saying that NB basically applies the Bayesian rule to calculate the probability or likelihood that a set of words (document) belongs to a class. To calculate this probability it assumes that all the words are independent or, in other words, the location of words in a sentence is completely random . (e.g., program is independent of enhancement) 

To learn more about Naive Bayes you can go to the links at the end of this notebook.

In [80]:
# import the model
from sklearn.naive_bayes import GaussianNB

# instantiate the class
nb = GaussianNB()

# get the 
y = df.loc[:, 'type']

In [82]:
scores = cross_val_score(nb, X, y, cv=3)
scores

array([0.65836299, 0.67544484, 0.6569395 ])

Just to show you how a more complex model would work, we'll take a look at Random Forest Classifier. However, a more powerful algorithm might take more time to converge and reach an answer. Run the following cell to see how it works.

In [83]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier()
scores = cross_val_score(random_forest, X, y, cv=3)
scores

array([0.78149466, 0.79003559, 0.78007117])

References:

https://www.kaggle.com/code/mitramir5/text-eda-rnn-lstm-gru-and-many-more


https://towardsdatascience.com/training-a-naive-bayes-model-to-identify-the-author-of-an-email-or-document-17dc85fa630a

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

